== 工具 == 
python3 (3.10+), seleniumwire, undetected_chromedriver


== 網站 ==
www.partslink24.com

== 說明 == 

1. 從入口網站左下方點選 bmw 徽標進入車型點選畫面
2. 從車型頁面源碼中找到相對應的車型 API 接口，進行呼叫。目的是要模擬點選
	X1 -> F48 -> SAV -> X1 18d -> THA > Engine
3. 每一個點擊都有相對應的接口，和 json 返回內容。通過這些 json 找到下一層的 API 並獲得 json 內容。


=== 第一题 ===
在 partslink 下，一個汽車品牌下有多個車型年款，由不同的參數組裝而成.

掃描獲得 BMW 下所有車型，並保留每一個接口對應的 json 內容
e.g. 
  https://www.partslink24.com/p5bmw/extern/groups/func-mdl?hg=04&lang=zh-TW&mdl=X1&modelType=F48&res1=SAV&res2=X1+18d&res3=THA&serviceName=bmw_parts


=== 第二题 ===

汽車品牌數據是樹狀結構。以BMW為例，

BMW - 1'
		- E81 
			- 3 Doors
				- 116d
					- ECE 
						- 11 Engine 
							- 11_4021 Short Engine 
								- Part 1
								- Part 2 
								
							- 11_4023 Engine block
							- 11_4099 Engine block mounting parts 
		- E82
		- E87

由於車型眾多，為了加速數據獲取，一般都會以分散式框架，平行獲取。
在第一題 我們知道n+1層數據的API 都可以在 第n層 的API裡獲得。

假設獲得的API URL 會在 30分鐘後過期，且有五隻爬蟲實例，每個爬蟲每次可以發送一個 URL 請求，
請問該如何規劃請求來達到最少重複請求(最少請求次數) 

In [19]:
from seleniumwire import webdriver
from undetected_chromedriver import Chrome, ChromeOptions
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from fake_useragent import UserAgent
import json

In [2]:
options = ChromeOptions()
# 设置User-Agent

In [3]:
# set options to run Chromedriver in headless mode
options.add_argument('--headless')
options.add_argument('--disable-gpu')

# set options to prevent Chrome from showing the "Chrome is being controlled by automated test software" notification
options.add_argument('--disable-logging')
options.add_argument('--disable-extensions')
options.add_argument('--disable-infobars')
options.add_argument('--disable-blink-features=AutomationControlled')

# set up user agent rotation
ua = UserAgent()
options.add_argument(f'user-agent={ua.random}')

# create a new undetected Chromedriver instance with the specified options
driver = webdriver.Chrome(options=options)

In [68]:
driver.get('https://www.partslink24.com/partslink24/user/login.do')

In [4]:
driver.get('https://www.partslink24.com/partslink24/launchCatalog.do?service=bmw_parts')

In [6]:
driver.page_source

'<html lang="en"><head>\n    <meta charset="utf-8" http-equiv="X-UA-Compatible" content="IE=10,11,edge">\n    <meta name="viewport" content="user-scalable=no, initial-scale=1, maximum-scale=1, minimum-scale=1, width=device-width, height=device-height, target-densitydpi=device-dpi">\n    <title>BMW, X5, E53, X5 4.8is – partslink24</title>\n    <link rel="stylesheet" href="p5-1.10.8.css">\n    <link rel="stylesheet" href="webfonts/font-style-1.10.8.css">\n    <link rel="stylesheet" href="webfonts/pl24-styleguide.1.2.11.min.css">\n    <link rel="stylesheet" href="galleria/galleria.classic.min.css">\n</head>\n<body>\n<div id="header" class="p5_header p5_demo">\n    <div id="brand_logo_container" class="p5_brand_logo_container">\n        <img id="brand_logo" src="data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAASwAAAEsCAYAAAB5fY51AAAACXBIWXMAAAsSAAALEgHS3X78AAAgAElEQVR42u2dT2wk133nv2oLIA8CyQSQZUF/2BqMtDEwMilAcAzBMmsOhjcXTQ+MtR35wKY2xmBzUU8ziA97mJ4A3mwW6FIPsnsQvDCbWWBynKb2kNXGgKqdwIeBFyKjsZyD

In [26]:
import gzip
for request in driver.requests:
    if request.response and 'models?' in request.url :
        content_type = request.response.headers.get('Content-Type', '')
        if 'json' in content_type:
            if 'gzip' in request.response.headers.get('Content-Encoding', ''):
                # 解压缩数据
                compressed_data = request.response.body
                decompressed_data = gzip.decompress(compressed_data)
                
                # 解码为字符串
                decoded_data = decompressed_data.decode('utf-8')
                
                # 解析为 JSON
                response_json = json.loads(decoded_data)
                print(request.url)
                print("Response JSON Data:", response_json)
                break

https://www.partslink24.com/p5bmw/extern/vehicle/models?lang=en&serviceName=bmw_parts&_=1692112021825
Response JSON Data: {'crumbs': [{'name': 'BMW', 'link': {'wid': 'modelTable', 'path': '/p5bmw/extern/vehicle/models?lang=en&serviceName=bmw_parts&upds=2023-07-13--12-55'}}], 'demo': True, 'data': {'records': [{'id': "1'", 'link': {'wid': 'modelTypeTable', 'path': '/p5bmw/extern/vehicle/modeltypes?lang=en&mdl=1%27&serviceName=bmw_parts&upds=2023-07-13--12-55'}, 'unavailable': True, 'values': {'model': "1'"}}, {'id': "2'", 'link': {'wid': 'modelTypeTable', 'path': '/p5bmw/extern/vehicle/modeltypes?lang=en&mdl=2%27&serviceName=bmw_parts&upds=2023-07-13--12-55'}, 'unavailable': True, 'values': {'model': "2'"}}, {'id': "3'", 'link': {'wid': 'modelTypeTable', 'path': '/p5bmw/extern/vehicle/modeltypes?lang=en&mdl=3%27&serviceName=bmw_parts&upds=2023-07-13--12-55'}, 'unavailable': True, 'values': {'model': "3'"}}, {'id': "4'", 'link': {'wid': 'modelTypeTable', 'path': '/p5bmw/extern/vehicle/mo

In [28]:
response_json['data']

{'records': [{'id': "1'",
   'link': {'wid': 'modelTypeTable',
    'path': '/p5bmw/extern/vehicle/modeltypes?lang=en&mdl=1%27&serviceName=bmw_parts&upds=2023-07-13--12-55'},
   'unavailable': True,
   'values': {'model': "1'"}},
  {'id': "2'",
   'link': {'wid': 'modelTypeTable',
    'path': '/p5bmw/extern/vehicle/modeltypes?lang=en&mdl=2%27&serviceName=bmw_parts&upds=2023-07-13--12-55'},
   'unavailable': True,
   'values': {'model': "2'"}},
  {'id': "3'",
   'link': {'wid': 'modelTypeTable',
    'path': '/p5bmw/extern/vehicle/modeltypes?lang=en&mdl=3%27&serviceName=bmw_parts&upds=2023-07-13--12-55'},
   'unavailable': True,
   'values': {'model': "3'"}},
  {'id': "4'",
   'link': {'wid': 'modelTypeTable',
    'path': '/p5bmw/extern/vehicle/modeltypes?lang=en&mdl=4%27&serviceName=bmw_parts&upds=2023-07-13--12-55'},
   'unavailable': True,
   'values': {'model': "4'"}},
  {'id': "5'",
   'link': {'wid': 'modelTypeTable',
    'path': '/p5bmw/extern/vehicle/modeltypes?lang=en&mdl=5%27&se

In [59]:
elements = driver.find_elements(By.CLASS_NAME, 'brand-logo')

In [60]:
for element in elements :
    if element.get_attribute('title') == 'BMW':
        button = element
        print('found it')
        break

found it


In [66]:
button.get_attribute('href')

'https://www.partslink24.com/partslink24/launchCatalog.do?service=bmw_parts'

In [67]:
button.click()

ElementNotInteractableException: Message: element not interactable
  (Session info: headless chrome=87.0.4280.67)
Stacktrace:
0   chromedriver                        0x0000000104534d79 chromedriver + 2510201
1   chromedriver                        0x0000000104ba1743 chromedriver + 9246531
2   chromedriver                        0x000000010436f4aa chromedriver + 652458
3   chromedriver                        0x00000001042f6be6 chromedriver + 158694
4   chromedriver                        0x00000001042ed52c chromedriver + 120108
5   chromedriver                        0x0000000104311c02 chromedriver + 269314
6   chromedriver                        0x00000001042ed2a5 chromedriver + 119461
7   chromedriver                        0x0000000104311e7e chromedriver + 269950
8   chromedriver                        0x0000000104320a96 chromedriver + 330390
9   chromedriver                        0x0000000104311e23 chromedriver + 269859
10  chromedriver                        0x00000001042eb00a chr